# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [3]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    '''
    Your code here
    
    提示：可參考以下的Stack Overflow: 
    https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
    '''

    # Scroll down to the bottom.
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load the page.
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")

    if new_height == last_height:

        break

    last_height = new_height

### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [4]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [5]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 「勞保退休金」該不該提前領？預期再6年破產...專家「4個字」給忠告

▲預期勞保將在2026年破產，剩下不到6年的時間，是否會掀起擠兌潮各界關注中。
圖文／鏡週刊
目前勞保超過55歲以上的被保險人約151萬餘人，他們都是未來5到10年內陸續退休的待退族，在距離勞保破產年限2026年，改革聲浪再起的當下，很多人在問，是不是應該提前領？
 
「我這個月去領了勞保老年給付，而且是一次請領，一共領了100多萬元。」50歲出頭的陳小姐因為投保年資和年齡符合資格，她說思考了很久，終於去領了，「因為我是在職業工會投保，保費要自付60%，如果想要按月領勞保年金，我得等到65歲才可以符合請領資格，這中間我還得再繳10多年的保費。」
陳小姐做決定的背後有很多考量因素，一方面是保費成本高，二方面是改革的不確定性，三方面是她有理財規劃，四方面則是她對國內政經環境有所不安。「我自己有在投資股票，這100多萬元，我準備好好的投資理財，放在殖利率好的標的上。」
▲年輕上班族擔心，到底勞保會不會破產？（圖／pakutaso）
到底勞保會不會破產？淡江大學風險管理與保險學系副教授郝充仁簡潔明快說了4個字：「相信政府」，他再以美股妙喻形容，「因為不信任而先領走老年給付，就如同是放空美股，這幾年美股一有危機，聯準會都會出手，勞保也是一樣，政府不會放任。」
陳小姐和莊太太的情況不同，決定也不同，而這其實也反應出每個勞工本身的年齡、投保薪資和年資、個人經濟條件、理財知識是否充足都不同。專精企管顧問精算師葉崇琦直言：「要不要先領？選一次領或月領？它沒有100分的答案。你不領，擔心勞保改革的風險，你領出來，也有投資虧損的風險啊！而絕大多數的勞工不是精於投資理財的人。」
葉崇琦強調，月領是活越久領越多，就算活不夠久也不吃虧，因為很多人忽略了勞保年金還有「遺屬年金」的機制，假設勞工領了2個月勞保年金後就死亡，他的遺屬可以選擇領走和一次領的差額（1,575,000元-32,550=1,542,450元），或者月領勞保年金折半的遺屬年金（16,275元除以2=8,138元），有未成年子女還有上限50％加給。
勞保老年年金給付金額A、B二式擇其高
每提早1年領，減給4%

文／中央社
汐止遠雄廣場購物中心今天下午發生一起5歲女童搭乘手扶梯時腳趾遭夾斷的意外。遠雄企業集團發言人楊舜欽回應，對於發生意外，實感遺憾，將全力協助女童與家屬處理後續事宜。
遠雄企業集團發言人楊舜欽指出，女童著布希鞋不慎捲入手扶梯，隨即緊急停止手扶梯運作，並立刻通知消防救護單位，工務人員配合拆卸，目前女童已送醫縫合。
楊舜欽說，布希鞋因鞋面寬、鞋底厚，很容易無意間太貼近梯間齒座，加上腳底與鞋面摩擦力小，鞋底與手扶梯面摩擦力大，孩童可能會反應不及而遭捲入。因此，賣場手扶梯旁立有告示牌，搭乘時須注意布希鞋與涼鞋的安全性。
楊舜欽表示，對於發生意外，相當遺憾，賣場員工全程陪同，並負責善後，將全力協助女童與家屬處理後續事宜。
----------------------------------------------------------------------

[9] 貝恩資本豪賭！接手「破產+負債1015億」維珍澳洲航空！

實習記者周依儒／綜合報導
新型疫情重擊航空業，澳州第二大航空維珍澳洲航空（Virgin Australia），4月底時便因長期債務問題宣布破產，進入破產管理程序尋求買主。美國私人股權投資公司貝恩資本（Bain Capital）也於今（26）日宣布將買下維珍航空，且交易將在8月底前完成。
根據CNN報導，貝恩資本擊敗了紐約的賽勒斯資本（Cyrus Capital），成為維珍航空的新主人。貝恩資本表示，他們將會替現有的營運團隊提出改善計劃，保住所有員工的工作，未來也會投入大量資金。
據悉，維珍航空的對手、澳州第一大航空澳洲航空（Qantas Airways）目前也已陷入困境，甚至在近日宣布將裁員6000人，且要透過股票集資19億澳幣（約385億新台幣）。而維珍航空在宣布破產前曾向澳州當局尋求幫助，但澳洲總理莫里森（Scott Morrison）卻表示，「我們希望澳州能有競爭激烈的航空市場，因此希望看到兩家航空公司能夠進行飛行競爭。」
國際航空運輸協會曾預估，全球航空業者大約需要3年的時間才能回復到疫情爆發前的水平。不過維珍航空在疫情爆發前就已經負債50億澳幣（約1015億新台幣），因此也有人指出，貝恩資本在此時收購景氣極差的航空業是一場「豪賭」。
--------------------------------------------------

▲人力銀行指出，上班族投上百封履歷卻石沉大海。（圖／取自免費圖庫Pixabay）
記者余弦妙／台北報導
新冠肺炎疫情在台灣逐漸趨緩，隨著後防疫時代來臨，消費市場似乎也開始逐步回溫，不過根據主計總處公布最新的失業率為4.07％創8年以來同期新高，不少上班族也開始受到衝擊，根據1111人力銀行指出，不少上班族在求職過程中也受到不少阻礙。
1111人力銀行表示，25歲的Neil（化名）原本是在遊戲公司上班，月薪為3萬8千元，但因車禍受傷只好辭去工作休養3個月，在今年2月身體康復，打算重返職場，卻因為疫情而打亂求職進度，很多企業幾乎不找人，Neil雖然投出近百封履歷態度十分積極，卻仍不敵景氣影響，至今仍待業中。
Neil表示，去年2月在找工作時，平均每天都會收到20多家的面試邀約，但今年很明顯的工作變難找了，從2月起到現在已投出不下百封履歷，有回覆的多半是保險、房仲及保全類的工作，完全不符他的求職期待，即使有藥局邀他去面試櫃台，也只開出2萬8千元待遇，和前一份工作相比明顯少了1萬元，自然無法屈就。
1111人力銀行媒體中心總經理何啟聖表示，從1111人力銀行近三個月的資料庫數據顯示，3月份疫情最嚴峻的時期，經濟表現受到明顯衝擊，因為業務緊縮使得就業市場的人力需求銳減，3到4月之間工作機會數明顯少了9%，而隨著國內防疫成效顯著，社區安全受到控制，加上政府積極提振國內觀光旅遊市場，帶動餐飲旅遊娛樂相關產業業績翻升，因而從4月到5月間的工作機會數也小幅攀升5%。
►過期票券別浪費！點這邊...幫你搶救變身
----------------------------------------------------------------------

[21] 以房養老4大祕訣讓你享清福   1800萬房每月可領3.5萬

▲隨著高齡化、少子化的趨勢，高齡者愈來愈重視「自己養自己」的議題，以房養老就是方式之一（圖／記者徐文彬攝）
記者吳靜君／台北報導
面對高齡化、少子化社會，愈來愈多人擔心退休金不足，或者必須「自己養活自己」的問題，因此政府與銀行業者也推動「以房養老」商品，讓退休族群靠房子養活自己，不僅高齡者有居住的地方，又可以有錢過生活。應該要如何選擇對自己有利的以房養老商品，銀行透露四大祕訣，讓退休族可以輕鬆享受樂活的晚年。
根據金管會統計，以房養老自2015年底開辦以來，到今年

►過期票券別浪費！簡單3步驟　教你搶救妙招
----------------------------------------------------------------------

[29] 理財周刊／低基期產業　剛從谷底復甦

汽車零組件族群壓抑已久，當疫情改變使用大眾運輸的習慣，自用車需求增加，相關低幾期個股籌碼經長期沉澱，有機會自谷底翻升。
文／馮泉富
台股加權指數自6月11日的長黑之後就陷入膠著，截至6月19日止未再進一步下跌，也沒有任何突破。美股S&P500走勢也雷同，唯獨代表科技股的NASDAQ期貨指數相對強勢，隨時有創高的可能，不過，若剔除尖牙股，走勢就沒這麼亮麗了。
全球股市不缺資金　短暫回調有利後市
現階段市場資金的追價力道趨緩，主因在於實體經濟修復的速度跟不上股市腳步，資金的風險偏好降低也屬正常。觀察美國的經濟現況，眾所周知，消費占美國GDP的比重高達七成，而零售銷售更是占了消費中的四成，因此，零售銷售數據可以判斷美國整體消費輪廓。根據美國商務部公布的五月零售銷售額為4860億美元，較四月回升17.7％，比路透調查分析師預期的8％相比多出一倍以上。
然而，五月零售額與疫情爆發前的二月零售額5270億美元相比仍有一段距離，且比去年同期相比依舊下滑6.1％，是否能完全修復，仍需謹慎觀察。再觀察月增率大幅上揚的細項，各類零售品項當中，先前受到疫情重創的服飾與家具需求都在解封後強力反彈。例如，五月服飾零售額較四月成長將約188％，但與去年同期相比仍下滑63.4％；家具銷售月增89.7％，但與去年同期相比仍下滑21.5％。
另一方面，欲期待消費擴張，又得觀察就業概況，根據商務部數據，截至6月13日止的當周，初領失業救濟人數雖降至約150.8萬人，但連續請領失業救濟金的人數為2054.4萬人，仍舊是一個龐大的數字，顯然，待業人數下降的速度太慢，這將是經濟修復過程的一大隱憂。
面對股市回調  需時間與耐心
誠如上週舉的例子，基本面猶如主人，股價猶如活潑的狗。當狗跑得太前面，大概率會有兩種動作，一是停下來望著等主人，二是回頭跑向主人，這就是全球股市目前的狀態。手上握有聯邦公開市場委員會(FOMC)投票權的聖路易斯聯準銀行總裁布拉德(James Bullard)日前表示，疫情引起的經濟危機可能已在四月觸底，不過，美國經濟雖可能已度過最糟的時期，但實際數據仍遠

▲勞保局指出，天災停班日若想加、退保，利用線上服務就可報。（圖／記者李毓康攝）
記者余弦妙／台北報導
夏季來臨，接下來就會進入颱風旺季，勞保局指出今年5月開始，就開放勞保網路申辦單位可以在颱風、豪雨等天然災害停班的隔天申報，勞保局的網路服務系統就會追朔。
勞保局表示，以往有勞工在天然災害停班日到、離職，投保單位都需要透過書面方式向勞保局申請追朔員工在該日加、退保。由於有部分投保單位反映天然災害後部分交通阻斷，書面申請較不便，因此勞保局也立即規劃在勞保局e化服務系統，新增天然災害停班日追溯加、退保功能，提供更快捷、便利且安全的申報作業服務。
另外，勞保局進一步補充說明，近年也有利用數位科技提升服務品質，如今年1月起開放投保單位可於員工到、離職日前10天內於網路預約加、退保；5月開放天然災害停班之投保單位可於線上辦理追溯加、退保。
►過期票券別浪費！點這邊...幫你搶救變身
----------------------------------------------------------------------

[39] 財訊／中國沒賺錢台灣走下坡　誠品吳旻潔全通路計畫外界張大眼睛看

文／林苑卿
6月1日凌晨，台北市敦南誠品正式熄燈，這場堪稱誠品近年在台灣最大規模的品牌行銷系列活動，藝文名人雲集、粉絲擠爆現場、落淚不捨的畫面占據當天各大媒體版面，獲得廣大回響。然而，就在幾天之前的誠品生活股東會上，場子主人同樣都是董事長吳旻潔，但環境氛圍卻是截然不同。
中國投資虧損　賠最慘的深圳店何去何從？
「深圳店未來的方向，我們現在積極與房東（華潤置地）談徹底與根本的方案，主要就是從能夠提升財報的損益，與重新進行資源配置為目標。第3季如果有比較明確的方向，會再向投資人進行報告。」5月27日，吳旻潔眼前是帶著質疑的股東們，她必須針對去年在中國地區虧損最多的深圳店說明處置策略。
接著6月上旬，市場就傳出誠品生活深圳店年底將從華潤置地旗下的華潤萬象天地撤場。對於《財訊》的查證，誠品生活回應：「誠品生活深圳正常營運，目前沒聽說有任何變動的計畫。」
儘管如此，但數字會說話，攤開誠品生活歷年的財報，前進中國根本是賺少虧多。從2012年開始，誠品生活在中國持續虧損了5年，2017年好不容易轉盈，且小幅成長，好光景就沒了，2018年2019年分別是衰退和虧損。而主要的虧損，就是來自深圳店和上海

...

圖文／鏡週刊
目前唯一合法可以管理頴川建忠千億財產的台北市政府社會局，2018年在法官裁定「輔助宣告後」，接手這件史無前例的千億元財產監管案，過程戰戰兢兢，深怕一個不小心，也會成為頴川家族爭產下的被告。
 
由於頴川的資產龐大，加上遍布海內外，子女間又有不同意見，社會局不想捲入雙方的爭奪戰，已向台北地院表達希望另請高明，由法院選定律師或會計師等專業人員管理，此案目前由台北地院家事庭審理中。
頴川建忠的5個子女與老臣，為了爭產已分裂成2派，次子頴川欽和陣營反對由哥哥頴川浩和派的任何人擔任管理父親財產的輔助人，基於雙方無法共同推舉中立、無利害關係的第三人擔任頴川建忠的輔助人，法院決定依「當事人最大利益」原則，2018年2月裁定由台北市社會局當輔助人，協助管理頴川建忠的財產，但仍無法平息二派糾紛。
頴川建忠的子女2016年9月帶著父親到台大醫院鑑定，經醫師認定，頴川建忠已屬「中度失智」，長子頴川浩和於是向台北地院聲請對父親「輔助宣告」，希望透過法律程序，請求法官幫已失智的父親決定把資產管理權交給自己或老臣、同時擔任大洋僑果公司及味王食品董座的陳清福，擔任資產輔助人，台北地院已裁定由台北市社會局擔任頴川建忠的輔助人。
知情人士說，當年法律制度不健全，若有現在的「意定監護」制度，就可以在自己仍清醒時，透過公證等方式，選定將來失智失能後的監護人管理財產。如今，對於頴川建忠當年登記給3個兒子及老臣陳清福的相關股權，究竟是借名登記、還是接班布局？不僅無法由他親自釐清，連法院恐怕也很難給子女們皆大歡喜的答案，連帶影響的是民視及味王的經營權，恐也隨時可能因法院的認定而波瀾再起。
►夏日想要清爽一點，快買神水
----------------------------------------------------------------------

[54] 日月光投控樂觀看下半年景氣　吳田玉感慨：員工健康與免疫力才是最重要！

▲日月光投控營運長吳田玉股東會後受訪。（圖／記者楊絡懸攝）
記者楊絡懸／高雄報導
半導體封裝大廠「日月光投控」（3711）營運長吳田玉股東會後表示，今年上半年的表現可圈可點，受惠於客戶轉單效應，營收和獲利可望較預期佳，相當感謝台灣的防疫工作，讓日月光投控台灣廠區上半年未受停工影響。
日月光投控股東會於高雄楠梓加工出口區舉行，因應新冠肺炎（COVID-19）疫情

▲鴻海土城總部。（圖／記者姚惠茹攝）
記者姚惠茹／台北報導
鴻海（2317）今（24）公告，今年每股配發現金股利4.2元，訂定除息交易日為7月23日，並預計將股利發放日為8月21日，今年總計將發出582.25億元的現金股利。
鴻海公告，除息基準日訂於7月31日，普通股最後過戶日為7月26日，停止過戶期間為7月27日至31日，海外可轉換公司債停止轉換期間為7月6日至7月31日。
鴻海今年每股配發現金股利4.2元，若以今日收盤價83.5元計算，現金殖利率達5%，而鴻海創辦人郭台銘目前持有鴻海股票134.17萬張，預計將能進帳56.35億元的現金股利，至於鴻海董事長劉揚偉則持股656張，預計可領到276萬元的現金股利。
►過期票券別浪費！簡單3步驟　教你換1.5倍東森幣
----------------------------------------------------------------------

[63] 鴻準董事長洪誌謙辭任　由李光曜接任新董座

▲鴻海旗下鴻準人事異動。（圖／路透）
記者姚惠茹／台北報導
鴻海旗下鴻準（2354）今（24）日公告董事長洪誌謙辭任，並經董事會推舉李光曜以鴻揚創業投資股份有限公司代表人身分擔任董事長，新人事異動即日起生效。
鴻準董事會同時決議將在7月23日除息，每股配發現金股利2.5元，若以今日收盤價55.9元計算，現金殖利率達4.5%，並預計8月21日發放現金股利。
►過期票券別浪費！簡單3步驟　教你換1.5倍東森幣
----------------------------------------------------------------------

[64] 統一發票四獎4000元可直接存入信用卡　7月底上路！　

▲ 隨著財政部調高統一發票中獎獎金免稅額度，信用卡載具可存獎金也隨之提高（圖／記者劉維榛攝）
記者吳靜君／台北報導
財政部去（2019）年12月放寬彩券獎金免稅門檻從2000元到5000元，財政部近期也發布函釋，未來雲端發票的四獎4000元獎金也可以存入、直接匯入信用卡載具，預計7月26日正式生效，財政部官員表示，依據生效日期，7、8月期後的發票可以適用。
財政部目前只有五獎（1000元）、六獎（200元）以及雲端發票千元獎、五百元以及百元都可以直接匯入信用卡內；但是去年財政部鬆綁民眾購買政府舉辦的獎

記者陳心怡／台北報導
證交所今（24）日舉辦股東會，總經理簡立忠指出，公司會持續進行改革，例如實施逐筆交易制度、規劃盤中零股交易、開放證券商轉投資私募股權基金等，期使交易制度更健全、證券市場更活絡。另外，也在會中通過配發股利2.7元。
簡立忠表示，受惠於市場資金寬鬆，全球主要股市多呈上漲趨勢，臺股大盤指數去年底為11,997.14點，較107年底上漲23.33%。上市證券全年平均日成交金額達新臺幣1,200億元，雖較107年減少7.79%，仍是近12年次高。
展望未來，簡立忠指出，在發行市場方面，公司將持續推動國內外優良企業上市，開拓新南向地區及潛力目標市場；此外，輔導上市公司深化公司治理及設置公司治理主管、推動董事會績效評鑑、研議修正機構投資人盡職治理守則等。
在交易市場方面，簡立忠提到，於今年3月23日起順利實施盤中全面逐筆交易，盤中零股交易則預計於今年10月26日實施；另因應市場需求，持續調整交割結算基金制度、研議低流動性股票造市制度、籌劃放寬有價證券借貸期限及以外幣作為擔保品等措施，期使證券市場更加蓬勃發展。
在商品方面，簡立忠表示，證交所將持續開發多元新商品，發展新類型ETF、新權證商品，並協助主管機關推動調整REITs之法制與稅制作業；另外，亦將持續辦理投資論壇，強化新商品之推廣教育，擴大市場參與。
另外，證交所在股東會通過108年度盈餘將配發2.7元股利，其中現金股利2.4元、股票股利0.3元，為近10年來第一高股利分派。
►過期票券別浪費！簡單3步驟　教你搶救妙招
----------------------------------------------------------------------

[75] 疫情帶動芳療保健商機　佐登-KY業績大幅成長250%

▲佐登妮絲集團總經理暨營運執行長陳佳琦。（圖／佐豋-KY提供）
記者姚惠茹／台北報導
佐登-KY（4190）今（24）日宣布，因應新冠疫情持續擴散全球，集團快速轉以精油產品系列銷售為主軸，搭配線上直播帶動買氣，帶動今年1~5月的精油銷售業績比去年同期增加250%，未來可望為集團增加新的營收動能。
佐登-KY表示，集團攜手中興大學進行產學合作、法國知名精油生產廠商HRF簽約調配專屬的精油配方，並經研究實驗發現天竺葵、檸檬精油可望提升防禦力，雖仍須進一步人體試驗，但疫情爆發至今，集團上半

▲瑞銀將蘋果的目標價提高至400美元。圖為蘋果開發者大會WWDC,iOS 14。（圖／取自YouTube／Apple）
記者林鼎閎／綜合報導
由於6月份業績優於預期以及隨著新冠肺炎復甦的加速，瑞銀在最新出具的報告中，將蘋果的目標價提高至400美元。
瑞銀（UBS）分析師阿庫裡（Timothy Arcuri）表示，「該公司的供應鏈現況表明，6月份iPhone SE的表現非常好」。
分析師將6月iPhone的出貨量預估從3000萬支提高至3400萬支。瑞銀集團預計在12月的季度中，新iPhone的初始供應量約為7000萬支，到2020年第四季度的出貨量約為6500萬支。
Arcuri預測「隨著消費者支出從休閒，零售轉向技術，對穿戴設備的需求將強勁恢復到疫情前。」
但在市場普遍看漲的時候，高盛 (Goldman Sachs) 卻有不同的看法。高盛認為，蘋果的高價產品可能反過來削弱市場需求，因此將蘋果股票維持「賣出 (Sell)」評級，目標價為 263 美元。
高盛分析師 Rod Hall 雖然對於蘋果即將推出搭載自家晶片的 Mac 電腦、最新 iOS 14 系統在主畫面的改良，以及 iPadOS 14 全新「隨手寫」(Scribble) 等功能更新計畫表示驚艷，但他對於蘋果的後市卻沒那麼樂觀。
Hall 擔憂的點在於，蘋果的高價產品策略可能導致需求大幅低於市場預期，因此將蘋果股價維持「賣出」評級，目標價維持於 263 美元。蘋果23日收盤價為366.53美元，當日上漲2.13％。
----------------------------------------------------------------------

[84] 快訊／用電量創歷史六月用電新高　台電：台中電廠今晚點火供電

▲圖為台中電廠。（圖／台電提供）
記者林淑慧／台北報導
用電再飆破紀錄！台電指出，今(24)日因各地高溫炎熱，最高用電量出現在下午14點15分，達到3716.95萬瓩，不僅創下今年及歷史6月份用電量新高，更一舉衝上歷年用電量排名第3名。台電表示，台中電廠2號機今晚將點火，加入供電行列。
台電表示，儘管今日用電量創新高，備轉容量仍有408.65萬瓩，備轉容量率約10.99%，維持供電充裕的綠燈。此外，在用電量最高當下，太陽光電發電量也達到209.5萬瓩，穩定挹注尖峰用電。
台電強調，中

▲桃園小檜溪重劃區，憑著水岸綠意和地段優勢成為房市竄起新秀，其中『遠雄青溪』買氣最旺，公開近5個月即將圓滿完銷。（採訪撰稿／記者胡至欣；攝影剪輯／記者楊詩益）
記者胡至欣／桃園報導
桃園在2014年正式升格直轄市後，淨遷入人口連續6年居六都之冠，主要因桃園房價比雙北親民，年輕人負擔得起，另外交通建設逐步到位以及生活成本較低等利多，也是一大誘因。
財經專家阮慕驊認為，桃園市境內的快速道路、高架道路四通八達，又有桃園國際機場以及機場捷運線，再加上目前正在興建中的桃園捷運，交通設施相當完善，另外在整個生活環境上面，桃園物價也比台北低，都是吸引移居客的有利條件。
雙北客在預算考量下，轉往1小時生活圈的桃園購屋，尤其總價低、新屋供給量大的各重劃區最受青睞，而去年才開發完成的小檜溪重劃區，更憑著水岸綠意和地段優勢成為竄起新秀！其中主打30坪到45坪，合車位總價落在1,100萬到1,700萬間的『遠雄青溪』買氣最旺，公開近5個月即將圓滿完銷。
遠雄房地產行銷協理蔡紫祺指出，小檜溪重劃區未來將有捷運綠線G09站，可串連桃園火車站與機場捷運線，聯外交通便利，還有舊市區和「桃園新東區」大業大有商圈的生活機能加持，客戶住進來不僅能享有一個好的生活環境與區段，『遠雄青溪』更提供一條龍式服務讓人住得安心又安全。
腹地近40公頃的小檜溪重劃區，位於桃園站前區北側，被4大成熟商圈包圍，除了大業大有商圈，還有中正商圈、站前商圈以及藝文商圈，生活機能和學區都相當完善，能免去一般重劃區為人詬病的開發空窗期，加上區內有5座親水公園、綠地及溪岸親水步道，整體綠覆率約15%，讓住戶在城市中也能擁抱綠意，成功吸引北北桃置產客，其中也有不乏在桃機上班的航空從業人員。
「之前台北桃園通勤會太累，下班回家還要一段時間，所以就想說直接在桃園買房，重劃區的預售屋在分期部分，頭期款負擔會比較少一點。」年輕夫妻檔劉先生和太太，花了3個月的時間看遍桃園各重劃區，最後選定『遠雄青溪』，除了因為地段緊鄰成熟商圈，食衣住行育樂一應俱，兩人也坦言自己有品牌情節，「遠雄是大牌子，所以在保固上比較安心，服務或交屋之後有什麼問題，比較不怕找不到人！」
不只產品夠有力，『遠雄青溪』更靠著品牌力圈粉無數，而為了滿足不同需求的客群，遠雄將於小檜溪重劃區推出第二期，以中大坪數作主力，讓更多自住客、換屋族，也能享受離塵不離城的質感生活。
-----